In [51]:
from mass_sight.khipu import *
import numpy as np
from scipy.optimize import linear_sum_assignment
from sklearn.linear_model import HuberRegressor
import pandas as pd
import time

In [14]:
ds1 = khipu("data/ds1.csv", 
                  id_col=0, 
                  rt_col=2, 
                  mz_col=1, 
                  int_cols=(3, 4), 
                  delimiter=",", 
                  mode="pos", 
                  mz_tolerance_ppm=5, 
                  rt_tolerance=.05)
ds2 = khipu("data/ds2.csv", 
                  id_col=0, 
                  rt_col=2, 
                  mz_col=1, 
                  int_cols=(3, 4), 
                  delimiter=",", 
                  mode="pos", 
                  mz_tolerance_ppm=5, 
                  rt_tolerance=.05)

In [15]:
ds1

,Compound_ID,MZ,RT,Intensity,Annotation_ID,isotope,modification,khipu_id
0,QI12374,272.128010,2.021717,14281.887950,"(E,E)-Trichostachine",NaN,NaN,NaN
1,QI2606,197.067139,3.630750,66443.482760,"1,7-Dimethyluric acid",NaN,NaN,NaN
2,QI12180,347.221633,2.035100,4641.052642,11-Deoxycortisol,NaN,NaN,NaN
3,QI876,301.216087,1.855767,1899.868163,13-cis-Retinoic acid,NaN,NaN,NaN
4,QI13553,634.649270,1.650333,171463.842100,1-DeoxyCer 18:0;O2/24:1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
13356,QI9994,140.114905,5.536283,10.995634,NaN,NaN,NaN,NaN
13357,QI9995,470.350564,5.536283,4226.612171,NaN,NaN,NaN,NaN
13358,QI9996,698.511659,5.536283,32058.023720,NaN,NaN,NaN,NaN
13359,QI9997,382.440716,5.531817,228.105635,NaN,NaN,NaN,NaN


In [26]:
# Split khipu_id column and handle empty values
ds1['id'] = ds1['khipu_id'].str.split('_').str[0]
ds1['mz_group'] = [float(i) if pd.notna(i) else ds1.iloc[idx]['MZ'] - 1.007825 for idx, i in enumerate(ds1['khipu_id'].str.split('_').str[1])]

ds2['id'] = ds2['khipu_id'].str.split('_').str[0]
ds2['mz_group'] = [float(i) if pd.notna(i) else ds2.iloc[idx]['MZ'] - 1.007825 for idx, i in enumerate(ds2['khipu_id'].str.split('_').str[1])]

# for each NaN id, assign it a unique incremented id
# get max id from both datasets (ids are of form kp123)
all_valid_ids = pd.concat([ds1['id'].dropna(), ds2['id'].dropna()])
# Filter out any non-string values and extract numeric parts
valid_kp_ids = all_valid_ids[all_valid_ids.astype(str).str.startswith('kp')]
if len(valid_kp_ids) > 0:
    max_id = int(valid_kp_ids.str[2:].astype(int).max())
else:
    max_id = 0  # Default if no valid kp IDs found

# assign unique NaN ids with incremented values
ds1_na_mask = ds1['id'].isna()
ds2_na_mask = ds2['id'].isna()

# For ds1, assign unique incremented IDs starting from max_id + 1
ds1_na_count = ds1_na_mask.sum()
if ds1_na_count > 0:
    new_ids_ds1 = [f"kp{max_id + i + 1}" for i in range(ds1_na_count)]
    ds1.loc[ds1_na_mask, 'id'] = new_ids_ds1

# For ds2, assign unique incremented IDs also starting from max_id + 1
ds2_na_count = ds2_na_mask.sum()
if ds2_na_count > 0:
    new_ids_ds2 = [f"kp{max_id + i + 1}" for i in range(ds2_na_count)]
    ds2.loc[ds2_na_mask, 'id'] = new_ids_ds2


In [6]:
ds1_mz_khipus = (ds1.groupby('id')['mz_group'].unique())
ds1_mz_khipus_pd = pd.DataFrame({'id': ds1_mz_khipus.index, 'mz_groups': [x[0] for x in ds1_mz_khipus.values]})
ds2_mz_khipus = (ds2.groupby('id')['mz_group'].unique())
ds2_mz_khipus_pd = pd.DataFrame({'id': ds2_mz_khipus.index, 'mz_groups': [x[0] for x in ds2_mz_khipus.values]})

In [22]:
# for each khipu in ds1, find all khipus in ds2 that are within 5 ppm
# Calculate ppm difference between two m/z values
def ppm_diff(mz1, mz2):
    return abs(mz1 - mz2) / mz1 * 1e6

# Create empty list to store matches
matches = []

# For each row in ds1
for idx1, row1 in ds1_mz_khipus_pd.iterrows():
    mz1 = row1['mz_groups']
    
    # Find matches in ds2 within 5 ppm
    matches_mask = ds2_mz_khipus_pd['mz_groups'].apply(lambda x: ppm_diff(mz1, x) <= 5)
    matching_rows = ds2_mz_khipus_pd[matches_mask]
    
    # Add matches to list
    for idx2, row2 in matching_rows.iterrows():
        matches.append({
            'id1': row1['id'],
            'id2': row2['id'], 
            'mz1': row1['mz_groups'],
            'mz2': row2['mz_groups'],
            'ppm_diff': ppm_diff(row1['mz_groups'], row2['mz_groups'])
        })

# Convert matches to DataFrame
matches_df = pd.DataFrame(matches)


In [68]:
high_confidence_matches = matches_df.merge(
    ds1, left_on='id1', right_on='id', how='left'
    ).merge(
        ds2, left_on='id2', right_on='id', how='left'
    ).query('abs(RT_x - RT_y) < 0.5'
    ).query('(isotope_x == isotope_y)'
    ).query('(modification_x == modification_y)'
    ).assign(delta_MZ = lambda x: x.MZ_y - x.MZ_x
    ).assign(delta_RT = lambda x: x.RT_y - x.RT_x
    )

In [69]:
# get huber model for mz and rt
mz_huber = HuberRegressor().fit(high_confidence_matches[['MZ_x']], high_confidence_matches['delta_MZ'])
rt_huber = HuberRegressor().fit(high_confidence_matches[['RT_x']], high_confidence_matches['delta_RT'])

# get model coefficients
mz_huber_coef = mz_huber.coef_[0]
rt_huber_coef = rt_huber.coef_[0]

# get model intercept
mz_huber_intercept = mz_huber.intercept_
rt_huber_intercept = rt_huber.intercept_

ds2["RT_corrected"] = (ds2["RT"] - rt_huber_intercept)/(rt_huber_coef + 1)
ds2["MZ_corrected"] = (ds2["MZ"] - mz_huber_intercept)/(mz_huber_coef + 1)

In [94]:
final_matches = matches_df.merge(
    ds1, left_on='id1', right_on='id', how='left'
    ).merge(
        ds2, left_on='id2', right_on='id', how='left'
    ).query('abs(RT_x - RT_corrected) < 0.5'
    ).query('abs((MZ_corrected - MZ_x)/MZ_x * 1e6) < 10'
    ).query('(isotope_x == isotope_y) or (isotope_x.isna()) or (isotope_y.isna())'
    ).query('(modification_x == modification_y) or (modification_x.isna()) or (modification_y.isna())'
    ).assign(delta_MZ = lambda x: x.MZ_corrected - x.MZ_x
    ).assign(delta_RT = lambda x: x.RT_corrected - x.RT_x
    )

# Find compound IDs with multiple matches
duplicate_matches = final_matches.groupby('id1').size().reset_index(name='count')
duplicate_matches = duplicate_matches[duplicate_matches['count'] > 1]

In [95]:
# Initialize list to store best matches
best_matches = []

# For each compound with multiple matches
for compound_id in duplicate_matches['id1']:
    # Get all matches for this compound
    compound_matches = final_matches[final_matches['id1'] == compound_id].copy()

    # Calculate RT and MZ differences using corrected values
    compound_matches['rt_diff'] = abs(compound_matches['RT_x'] - compound_matches['RT_corrected'])
    compound_matches['mz_diff'] = abs(compound_matches['MZ_x'] - compound_matches['MZ_corrected'])
    
    # Score each match based on RT and MZ differences
    # Lower score is better
    compound_matches['match_score'] = compound_matches['rt_diff'] + compound_matches['mz_diff']
    
    # Get the match with the lowest score
    best_match = compound_matches.loc[compound_matches['match_score'].idxmin()]
    
    # Add to best matches list
    best_matches.append({
        'id1': best_match['id1'],
        'id2': best_match['id2']
    })

# Create DataFrame of best matches
best_matches_df = pd.DataFrame(best_matches)

# Combine best matches with non-duplicate matches
final_matches = pd.concat([
    final_matches[~final_matches['id1'].isin(duplicate_matches['id1'])],
    best_matches_df
])


In [1]:
from mass_sight.core import mass_combine

final_matches = mass_combine(
    ds1_file="data/ds1.csv",
    ds2_file="data/ds2.csv",
    id_col=0,
    rt_col=2,
    mz_col=1,
    int_cols=(3, 4),
    delimiter=",",
)

/Users/chiraag/Projects/gwu/lab/massSight/.venv/lib/python3.13/site-packages/khipu/model.py:143: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(_func, X, Y)


In [2]:
final_matches

,id1,id2,mz1,mz2,ppm_diff,Compound_ID_x,MZ_x,RT_x,Intensity_x,Annotation_ID_x,...,Annotation_ID_y,isotope_y,modification_y,khipu_id_y,id_y,mz_group_y,RT_corrected,MZ_corrected,delta_MZ,delta_RT
19,kp1337,kp1452,300.208262,300.208432,0.566274,QI876,301.216087,1.855767,1.899868e+03,13-cis-Retinoic acid,...,13-cis-Retinoic acid,NaN,NaN,NaN,kp1452,300.208432,1.877271,301.215871,-0.000217,0.021504
20,kp1338,kp1453,633.641445,633.642248,1.267278,QI13553,634.649270,1.650333,1.714638e+05,1-DeoxyCer 18:0;O2/24:1,...,1-DeoxyCer 18:0;O2/24:1,NaN,NaN,NaN,kp1453,633.642248,1.655823,634.649272,0.000002,0.005489
69,kp1340,kp1610,147.088910,147.089103,1.312132,QI5545,148.096735,7.945800,1.809700e+03,2-amino-6-hydroxyhexanoic acid,...,O-Ethylhomoserine,NaN,NaN,NaN,kp1610,147.089103,7.549868,148.096732,-0.000003,-0.395932
110,kp1343,kp1457,145.109692,145.109866,1.195647,QI6004,146.117517,8.878067,1.152901e+06,3-Dehydroxycarnitine,...,3-Dehydroxycarnitine,NaN,NaN,NaN,kp1457,145.109866,8.891973,146.117497,-0.000020,0.013906
128,kp1346,kp1460,133.073415,133.073635,1.648714,QI8221,134.081240,7.945800,9.290959e+03,3-hydroxynorvaline,...,3-hydroxynorvaline,NaN,NaN,NaN,kp1460,133.073635,7.929860,134.081281,0.000041,-0.015940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3571,kp999,kp1425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3572,kp9990,kp9913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3573,kp9996,kp9917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3574,kp9997,kp11470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
